# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [4]:
import pandas as pd
import numpy as np
from scipy import stats

In [5]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [6]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [7]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

# 1- What test is appropriate for this problem? Does CLT apply?

# 2- What are the null and alternate hypotheses?

# 3- Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [8]:
w = data[data.race=='w']
b = data[data.race=='b']

In [9]:
mean_w_obs=len(w.call[w.call==1])/len(w)
mean_b_obs=len(b.call[b.call==1])/len(b)
print('Rate of callbacks for whites: ', mean_w_obs)
print('Rate of callbacks for blacks: ', mean_b_obs)

Rate of callbacks for whites:  0.09650924024640657
Rate of callbacks for blacks:  0.06447638603696099


In [10]:
df=data[['race', 'call']]

In [11]:
df_b=data[['race', 'call']][data.race=='b']

In [12]:
df_w=data[['race', 'call']][data.race=='w']

In [54]:
calls=np.sum(df_w.call[df_w.call==1])
print(calls)

235.0


In [14]:
print('White - 1: ', len(df_w.call[df_w.call==1]))
print('White - 0: ', len(df_w.call[df_w.call==0]))
print('Black - 1: ', len(df_b.call[df_b.call==1]))
print('Black - 0: ', len(df_b.call[df_b.call==0]))

White - 1:  235
White - 0:  2200
Black - 1:  157
Black - 0:  2278


## 3.1  Frequentist  approache

   ### 3.1.1- Margin of Error 

In [15]:
# Your solution to Q3 here
#1==w
#2==b

# Margin of Error == z statistic * std 
#                 == (difference of means / sqrt(Var_t)) * sqrt(Var_1 + Var_2)
#                 == ((p1-p2) / sqrt([p*(1-p)/n1]+[p*(1-p)/n2])) * sqrt([p1*(1-p1))/n_w] + [p2*(1-p2)/n_b]) 

In [57]:
p1=mean_w_obs
p2=mean_b_obs
p=len(data.call[data.call==1])/len(data)
n1=len(w)
n2=len(b)
mean_diff=p1-p2
z_score=1.96 # z_score for 95% confidence interval

In [48]:
std=np.sqrt((p1*(1-p1)/n1)+(p2*(1-p2)/n2))
moe=z_score*std
print(moe)

0.015255406349886438


### 3.1.2- Confidence Interval 

In [50]:
# ci = confidence interval
# ci_lower = diff_means(p1-p2) - moe
# ci_upper = diff_means(p1-p2) + moe

In [52]:
ci_lower=(p1-p2)-moe
ci_upper=(p1-p2)+moe
print('Confidence interval: {}\t-\t{}'.format(ci_lower, ci_upper))

Confidence interval: 0.016777447859559147	-	0.047288260559332024


### 3.1.3- P value 

In [47]:
z_stat=(p1-p2)/(np.sqrt((p*(1-p)/n1)+(p*(1-p)/n2))) 
p_value=2*(1-stats.norm.cdf(abs(z_stat)))
print(p_value)

3.983886837577444e-05


### 3.1.4 Result 

## 3.2-  Bootstrapping approach

In [59]:
# Construct arrays of data: white-sounding names, black-sounding names
all_callbacks = np.array([True] * int(calls) + [False] * int(len(data)-calls))

size = 10000

bs_reps_diff = np.empty(size)

for i in range(size):
    w_bs_replicates = np.sum(np.random.choice(all_callbacks, size=len(w)))
    b_bs_replicates = np.sum(np.random.choice(all_callbacks, size=len(b)))
    
    bs_reps_diff[i] = (w_bs_replicates - b_bs_replicates)/len(b)
    
bs_p_value = np.sum(bs_reps_diff >= mean_diff) / len(bs_reps_diff)

bs_ci = np.percentile(bs_reps_diff, [2.5, 97.5])
bs_mean_diff = np.mean(bs_reps_diff)

print('obs diff: {}\n'.format(mean_diff))
print('BOOTSTRAP RESULTS\np-value: {}\n95% conf. int.: {}'.format(bs_p_value, bs_ci))

obs diff: 0.032032854209445585

BOOTSTRAP RESULTS
p-value: 0.0
95% conf. int.: [-0.01190965  0.01232033]


### Bootstraping Result:

We generated 10000 means for each group and there is no mean is smaller than the current one and therefore we found that p_value is less than 1/10000. So we need the reject Null Hypothesis.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

# 4- Write a story describing the statistical significance in the context or the original problem.

# 5-

Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?